<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc" style="margin-top: 1em;"><ul class="toc-item"><li><ul class="toc-item"><li><span><a href="#(a)" data-toc-modified-id="(a)-0.1"><span class="toc-item-num">0.1&nbsp;&nbsp;</span>(a)</a></span></li><li><span><a href="#(b)" data-toc-modified-id="(b)-0.2"><span class="toc-item-num">0.2&nbsp;&nbsp;</span>(b)</a></span></li><li><span><a href="#(c)" data-toc-modified-id="(c)-0.3"><span class="toc-item-num">0.3&nbsp;&nbsp;</span>(c)</a></span></li><li><span><a href="#(d)" data-toc-modified-id="(d)-0.4"><span class="toc-item-num">0.4&nbsp;&nbsp;</span>(d)</a></span></li></ul></li></ul></div>

Use Broyden’s method to approximate solutions to the nonlinear systems in Exercise 1 using the following initial approximations $x^{(0)}$.

The nonlinear system continuous with ex10.2.5

In [1]:
import numpy as np
from numpy import linalg
from abc import abstractmethod
import pandas as pd
import math

pd.options.display.float_format = '{:,.8f}'.format
np.set_printoptions(suppress=True, precision=8)

TOR = pow(10.0, -6)
MAX_ITR = 150

In [2]:
class NewtonMethod(object):

    def __init__(self):
        return

    @abstractmethod
    def f(self, x):
        return NotImplementedError('Implement f()!')

    @abstractmethod
    def jacobian(self, x):
        return NotImplementedError('Implement jacobian()!')

    @abstractmethod
    def run(self, x):
        return NotImplementedError('Implement run()!')

## (a) 
$$\begin{align*}
4x_1^2 - 20 x_1 + \frac{1}{4} x_2^2 + 8 &= 0 \\
\frac{1}{2}x_1x_2^2+2x_1-5x_2+8 &= 0
\end{align*}$$

In [3]:
class Broyden(NewtonMethod):

    def __init__(self):
        super(NewtonMethod, self).__init__()

    def f(self, x):
        sol = np.zeros(len(x))
        sol[0] = 4 * pow(x[0], 2) - 20 * x[0] + pow(x[1], 2) / 4 + 8
        sol[1] = x[0] * pow(x[1], 2) / 2 + 2 * x[0] - 5 * x[1] + 8
        return sol

    def jacobian(self, x):
        jac = np.zeros(shape=(2, 2))
        jac[0][0] = 8 * x[0] - 20
        jac[0][1] = x[1] / 2
        jac[1][0] = pow(x[1], 2) / 2
        jac[1][1] = x[0] * x[1] - 5
        return jac

    def run(self, x):
        df = pd.DataFrame(columns=['x' + str(i + 1) for i in range(len(x))] + ['residual', 'actual-residual'])

        row = len(df)
        df.loc[row] = [xe for xe in x] + [np.nan, np.nan]
        A0 = self.jacobian(x)
        v = self.f(x)
        A = linalg.inv(A0)
        s = -A.dot(v)
        nx = x + s
        row = len(df)
        x = nx
        residual = linalg.norm(s, np.inf)
        df.loc[row] = [nxe for nxe in nx] + [residual, np.nan]

        for k in range(2, MAX_ITR):
            w = v
            v = self.f(x)
            y = v - w
            z = -A.dot(y)
            p = -s.transpose().dot(z)
            u = s.transpose().dot(A)
            A = A + 1 / p * np.outer((s + z), u)
            s = -A.dot(v)
            nx = x + s
            residual = linalg.norm(s, np.inf)
            x = nx

            row = len(df)
            df.loc[row] = [nxe for nxe in nx] + [residual, np.nan]
            if residual < TOR:
                break

        for i in range(len(df)):
            xk = np.array([df.loc[i][j] for j in range(len(x))])
            df.loc[i][3] = linalg.norm(x - xk, np.inf)
        
        print(self.f(x))
        return df

In [4]:
x0 = np.array([0, 0])
Broyden().run(x0).astype(np.float64)

[ 0.  0.]


,x1,x2,residual,actual-residual
0,0.00000000,0.00000000,nan,2.00000000
1,0.40000000,1.60000000,1.60000000,0.40000000
2,0.47653348,1.91378728,0.31378728,0.08621272
3,0.49789847,1.99045549,0.07666821,0.00954451
4,0.50012064,2.00044219,0.00998670,0.00044219
5,0.50000654,2.00002635,0.00041583,0.00002636
6,0.49999979,1.99999917,0.00002719,0.00000083
7,0.50000000,2.00000000,0.00000083,0.00000000


In [5]:
x0 = np.array([5, 5])
Broyden().run(x0).astype(np.float64)

[  0.36079133  14.9793706 ]


,x1,x2,residual,actual-residual
0,5.00000000,5.00000000,nan,4.39979633
1,4.60338983,2.47288136,2.52711864,1.87267769
2,4.61528599,0.97198023,1.50090113,0.37177656
3,4.80670975,-4.60759008,5.57957031,5.20779375
4,4.60323382,2.04619110,6.65378118,1.44598743
5,4.55800565,3.51853709,1.47234599,2.91833342
6,4.68167284,-0.16790683,3.68644392,0.76811050
7,4.91200151,-7.24826984,7.08036301,7.84847351
8,4.64973273,0.61339056,7.86166040,0.07182933
9,4.62738811,1.32111194,0.70772139,0.72090827


## (b) 
$$\begin{align*}
\sin(4\pi x_1 x_2)-2x_2-x_1 &= 0 \\
\left( \frac{4\pi-1}{4\pi} \right)(\exp^{2x_1}-\exp)+4\exp x_2^2 -2\exp x_1 &= 0
\end{align*}$$

In [6]:
class Broyden(NewtonMethod):

    def __init__(self):
        super(NewtonMethod, self).__init__()

    def f(self, x):
        sol = np.zeros(len(x))
        sol[0] = math.sin(4 * math.pi * x[0] * x[1]) - 2 * x[1] - x[0]
        sol[1] = ((4 * math.pi - 1) / (4 * math.pi)) * (math.exp(2 * x[0]) - math.e) + 4 * math.e * pow(x[1], 2) - 2 * math.e * x[0]
        return sol

    def jacobian(self, x):
        jac = np.zeros(shape=(2, 2))
        jac[0][0] = 4 * math.pi * x[1] * math.cos(4 * math.pi * x[0] * x[1]) - 1
        jac[0][1] = 4 * math.pi * x[0] * math.cos(4 * math.pi * x[0] * x[1]) - 2
        jac[1][0] = 2 * (4 * math.pi - 1) / (4 * math.pi) * math.exp(2 * x[0]) - 2 * math.e
        jac[1][1] = 8 * math.e * x[1]
        return jac

    def run(self, x):
        df = pd.DataFrame(columns=['x' + str(i + 1) for i in range(len(x))] + ['residual', 'actual-residual'])

        row = len(df)
        df.loc[row] = [xe for xe in x] + [np.nan, np.nan]
        A0 = self.jacobian(x)
        v = self.f(x)
        A = linalg.inv(A0)
        s = -A.dot(v)
        nx = x + s
        row = len(df)
        x = nx
        residual = linalg.norm(s, np.inf)
        df.loc[row] = [nxe for nxe in nx] + [residual, np.nan]

        for k in range(2, MAX_ITR):
            w = v
            v = self.f(x)
            y = v - w
            z = -A.dot(y)
            p = -s.transpose().dot(z)
            u = s.transpose().dot(A)
            A = A + 1 / p * np.outer((s + z), u)
            s = -A.dot(v)
            nx = x + s
            residual = linalg.norm(s, np.inf)
            x = nx

            row = len(df)
            df.loc[row] = [nxe for nxe in nx] + [residual, np.nan]
            if residual < TOR:
                break

        for i in range(len(df)):
            xk = np.array([df.loc[i][j] for j in range(len(x))])
            df.loc[i][3] = linalg.norm(x - xk, np.inf)
        
        print(self.f(x))
        return df

In [7]:
x0 = np.array([0, 0])
Broyden().run(x0).astype(np.float64)

[ 0.  0.]


,x1,x2,residual,actual-residual
0,0.00000000,0.00000000,nan,0.37369822
1,-0.43984123,0.21992062,0.43984123,0.16365413
2,-0.32500698,-0.08035291,0.30027353,0.13661940
3,-0.32264114,0.07575868,0.15611159,0.05105708
4,-0.36088182,0.04396699,0.03824068,0.01281640
5,-0.37605752,0.05372155,0.01517570,0.00254494
6,-0.37371600,0.05627841,0.00255687,0.00001778
7,-0.37369782,0.05626618,0.00001818,0.00000040
8,-0.37369822,0.05626649,0.00000040,0.00000000


In [8]:
x0 = np.array([-10, -10])
Broyden().run(x0).astype(np.float64)

[      0.30178715  585361.90868364]


,x1,x2,residual,actual-residual
0,-10.00000000,-10.00000000,nan,453.00796223
1,-15.17335965,-4.63217129,5.36782871,447.83460258
2,-16.93031424,-2.63390287,1.99826842,446.07764799
3,-18.57201368,-0.57914347,2.05475940,444.43594855
4,-20.86067939,2.41664282,2.99578629,442.14728284
5,-15.82764260,-3.96175160,6.37839442,447.18031963
6,-31.81248157,16.30382280,20.26557440,431.19548066
7,-14.38655679,-5.55063902,21.85446181,448.62140544
8,-11.71330242,-8.64958241,3.09894339,451.29465981
9,-16.56582098,-2.84211879,5.80746362,446.44214125


In [9]:
x0 = np.array([1, -10])
Broyden().run(x0).astype(np.float64)

[-0.        0.537908]


,x1,x2,residual,actual-residual
0,1.00000000,-10.00000000,nan,9.46700018
1,1.56845109,-4.98388580,5.01611420,4.45088598
2,1.81173583,-3.23555010,1.74833571,2.70255027
3,2.06031647,-1.58245200,1.65309809,1.71045284
4,2.29545340,0.16192887,1.74438087,1.94558977
5,-1.08767165,-29.58926640,29.75119527,29.05626658
6,2.28993499,0.40128835,29.99055475,1.94007136
7,2.28192908,0.64413213,0.24284378,1.93206545
8,2.70311609,-13.13707145,13.78120358,12.60407163
9,2.22550813,1.18598420,14.32305565,1.87564450


In [10]:
x0 = np.array([-1, 1])
Broyden().run(x0).astype(np.float64)

[-0. -0.]


,x1,x2,residual,actual-residual
0,-1.00000000,1.00000000,nan,0.94373351
1,-2.02974173,0.11368626,1.02974173,1.65604351
2,-6.71789489,-3.05765564,4.68815316,6.34419667
3,-1.70213546,0.37983196,5.01575943,1.32843724
4,-1.38328782,0.54608180,0.31884764,1.00958960
5,-9.64651909,-4.34896125,8.26323127,9.27282087
6,-1.10620083,0.69705827,8.54031826,0.73250261
7,-0.80158860,0.83842245,0.30461223,0.78215596
8,4.42720509,1.67968737,5.22879369,4.80090331
9,-0.79460527,0.75169039,5.22181036,0.69542390


## (c) 
$$\begin{align*}
x_1(1-x_1)+4x_2 &= 12 \\
(x_1 - x_2)^2 + (2x_2-3)^2 &= 25
\end{align*}$$

In [11]:
class Broyden(NewtonMethod):

    def __init__(self):
        super(NewtonMethod, self).__init__()

    def f(self, x):
        sol = np.zeros(len(x))
        sol[0] = x[0] * (1 - x[0]) + 4 * x[1] - 12
        sol[1] = pow(x[0] - x[1], 2) + pow(2 * x[1] - 3, 2) - 25
        return sol

    def jacobian(self, x):
        jac = np.zeros(shape=(2, 2))
        jac[0][0] = 1 - 2 * x[0]
        jac[0][1] = 4
        jac[1][0] = 2 * (x[0] - x[1])
        jac[1][1] = -2 * (x[0] * x[1]) + 4 * (2 * x[1] - 3)
        return jac

    def run(self, x):
        df = pd.DataFrame(columns=['x' + str(i + 1) for i in range(len(x))] + ['residual', 'actual-residual'])

        row = len(df)
        df.loc[row] = [xe for xe in x] + [np.nan, np.nan]
        A0 = self.jacobian(x)
        v = self.f(x)
        A = linalg.inv(A0)
        s = -A.dot(v)
        nx = x + s
        row = len(df)
        x = nx
        residual = linalg.norm(s, np.inf)
        df.loc[row] = [nxe for nxe in nx] + [residual, np.nan]

        for k in range(2, MAX_ITR):
            w = v
            v = self.f(x)
            y = v - w
            z = -A.dot(y)
            p = -s.transpose().dot(z)
            u = s.transpose().dot(A)
            A = A + 1 / p * np.outer((s + z), u)
            s = -A.dot(v)
            nx = x + s
            residual = linalg.norm(s, np.inf)
            x = nx

            row = len(df)
            df.loc[row] = [nxe for nxe in nx] + [residual, np.nan]
            if residual < TOR:
                break

        for i in range(len(df)):
            xk = np.array([df.loc[i][j] for j in range(len(x))])
            df.loc[i][3] = linalg.norm(x - xk, np.inf)
        
        print(self.f(x))
        return df

In [12]:
x0 = np.array([0, 0])
Broyden().run(x0).astype(np.float64)

[-0.          0.00000003]


,x1,x2,residual,actual-residual
0,0.00000000,0.00000000,nan,3.89438027
1,17.33333333,-1.33333333,17.33333333,14.87692819
2,-2.22172264,-4.51819587,19.55505597,8.41257614
3,-0.98084388,5.26762859,9.78582446,3.43724903
4,44.55140440,106.67000783,101.40237924,102.77562755
5,-0.49051248,4.84117855,101.82882928,2.94691762
6,-0.05052663,4.49832924,0.43998585,2.50693177
7,2.36764504,2.78069712,2.41817167,1.11368315
8,1.39299403,3.46778254,0.97465101,1.06341111
9,1.58539061,3.35421296,0.19239658,0.87101453


In [13]:
x0 = np.array([1, 5])
Broyden().run(x0).astype(np.float64)

[ 0.  0.]


,x1,x2,residual,actual-residual
0,1.00000000,5.00000000,nan,1.45640514
1,2.14285714,3.28571429,1.71428571,0.60866599
2,1.09132420,3.26027397,1.05153294,1.36508094
3,2.70311611,3.85491059,1.61179191,0.24671097
4,2.27098545,3.83942194,0.43213066,0.18541969
5,2.44255329,3.88954538,0.17156784,0.01385185
6,2.45739772,3.89460835,0.01484442,0.00099257
7,2.45640599,3.89437853,0.00099173,0.00000174
8,2.45640524,3.89438026,0.00000173,0.00000010
9,2.45640514,3.89438027,0.00000010,0.00000000


In [14]:
x0 = np.array([-1, 500])
Broyden().run(x0).astype(np.float64)

[ -0.89972419  74.73012125]


,x1,x2,residual,actual-residual
0,-1.00000000,500.00000000,nan,493.61340783
1,-260.65781151,198.24335863,301.75664137,264.99135985
2,222.60583733,293.33389420,483.26364884,286.94730203
3,"1,617.41839988",502.59905891,"1,394.81256256","1,613.08485154"
4,202.38867611,222.10303653,"1,415.02972377",215.71644436
5,183.55316404,182.20822771,39.89480882,179.21961570
6,130.94834941,95.94823261,86.25999510,126.61480107
7,100.12195368,56.33545216,39.61278045,95.78840534
8,72.91496338,25.81002999,30.52542217,68.58141504
9,52.30618082,4.56839796,21.24163203,47.97263248


## (d) 
$$\begin{align*}
5x_1^2-x_2^2 &= 0 \\
x_2 - 0.25(\sin x_1 + \cos x_2) &= 0
\end{align*}$$

In [15]:
class Broyden(NewtonMethod):

    def __init__(self):
        super(NewtonMethod, self).__init__()

    def f(self, x):
        sol = np.zeros(len(x))
        sol[0] = 5 * pow(x[0], 2) - pow(x[1], 2)
        sol[1] = x[1] - 0.25 * (math.sin(x[0]) + math.cos(x[1]))
        return sol

    def jacobian(self, x):
        jac = np.zeros(shape=(2, 2))
        jac[0][0] = 10 * x[0]
        jac[0][1] = -2 * x[1]
        jac[1][0] = -0.25 * math.cos(x[0])
        jac[1][1] = 1 + 0.25 * math.sin(x[1])
        return jac

    def run(self, x):
        df = pd.DataFrame(columns=['x' + str(i + 1) for i in range(len(x))] + ['residual', 'actual-residual'])

        row = len(df)
        df.loc[row] = [xe for xe in x] + [np.nan, np.nan]
        A0 = self.jacobian(x)
        v = self.f(x)
        A = linalg.inv(A0)
        s = -A.dot(v)
        nx = x + s
        row = len(df)
        x = nx
        residual = linalg.norm(s, np.inf)
        df.loc[row] = [nxe for nxe in nx] + [residual, np.nan]

        for k in range(2, MAX_ITR):
            w = v
            v = self.f(x)
            y = v - w
            z = -A.dot(y)
            p = -s.transpose().dot(z)
            u = s.transpose().dot(A)
            A = A + 1 / p * np.outer((s + z), u)
            s = -A.dot(v)
            nx = x + s
            residual = linalg.norm(s, np.inf)
            x = nx

            row = len(df)
            df.loc[row] = [nxe for nxe in nx] + [residual, np.nan]
            if residual < TOR:
                break

        for i in range(len(df)):
            xk = np.array([df.loc[i][j] for j in range(len(x))])
            df.loc[i][3] = linalg.norm(x - xk, np.inf)
        
        print(self.f(x))
        return df

In [16]:
x0 = np.array([0, 0])
Broyden().run(x0).astype(np.float64)

LinAlgError: Singular matrix

In [17]:
x0 = np.array([1, 1])
Broyden().run(x0).astype(np.float64)

[ 0.00000001 -0.        ]


,x1,x2,residual,actual-residual
0,1.00000000,1.00000000,nan,0.87875808
1,0.48024079,0.40120393,0.59879607,0.35899887
2,0.35023441,0.33287304,0.13000638,0.22899249
3,0.24431126,0.30002484,0.10592315,0.12306934
4,0.17835025,0.28453401,0.06596101,0.05710834
5,0.13896975,0.27533553,0.03938050,0.01772784
6,0.12444706,0.27187221,0.01452269,0.00320514
7,0.12146690,0.27115874,0.00298016,0.00022499
8,0.12124629,0.27110618,0.00022061,0.00000438
9,0.12124201,0.27110518,0.00000428,0.00000010


In [18]:
x0 = np.array([-1, -1])
Broyden().run(x0).astype(np.float64)

[ 0.00000001 -0.        ]


,x1,x2,residual,actual-residual
0,-1.00000000,-1.00000000,nan,1.21950013
1,-0.34332125,0.28339374,1.28339374,0.24515780
2,-0.32567373,0.13847158,0.14492216,0.22751028
3,-0.27544965,0.17140066,0.05022408,0.17728620
4,-0.18677046,0.19737549,0.08867919,0.08860700
5,-0.13285122,0.21111591,0.05391924,0.03468776
6,-0.10767317,0.21722185,0.02517804,0.00950972
7,-0.09946730,0.21918949,0.00820587,0.00130385
8,-0.09822268,0.21948599,0.00124462,0.00005922
9,-0.09816369,0.21950008,0.00005898,0.00000024


In [19]:
x0 = np.array([100, -100])
Broyden().run(x0).astype(np.float64)

[ 0.00000001 -0.        ]


,x1,x2,residual,actual-residual
0,100.00000000,-100.00000000,nan,100.21950013
1,44.36090886,-21.80454429,78.19545571,44.45907231
2,28.06579526,-0.30437991,21.50016438,28.16395871
3,23.33264603,-0.02728724,4.73314923,23.43080948
4,14.55782648,-0.60154490,8.77481955,14.65598993
5,3.47783307,1.14122370,11.07999341,3.57599652
6,5.96255179,0.07927615,2.48471872,6.06071524
7,5.09670455,0.10524045,0.86584724,5.19486800
8,4.43876201,-0.10816375,0.65794254,4.53692546
9,6.70362318,0.04814751,2.26486117,6.80178663
